In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving text_jp.txt to text_jp.txt


In [ ]:
file = open("text_jp.txt", "r", encoding= "utf8" )
lines =[]
for i in file:
  lines.append(i)
data =""
for i in lines:
  data = ' '.join(lines)

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”', '')
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of A Room With A View, by E. M. Forster This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using t'

In [ ]:
len(data)

387734

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer, open('token.pkl','wb'))
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 114, 105, 573, 4, 5, 115, 19, 5, 195, 46, 310, 1223, 1839, 33]

In [ ]:
len(sequence_data)

70167

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7866


In [ ]:
sequences = []
for i in range(3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)

print("The length of the sequences are : ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The length of the sequences are :  70164


array([[   1,  114,  105,  573],
       [ 114,  105,  573,    4],
       [ 105,  573,    4,    5],
       [ 573,    4,    5,  115],
       [   4,    5,  115,   19],
       [   5,  115,   19,    5],
       [ 115,   19,    5,  195],
       [  19,    5,  195,   46],
       [   5,  195,   46,  310],
       [ 195,   46,  310, 1223]])

In [ ]:
x = [] 
y = [] 
for i in sequences:
  x.append(i[0:3])
  y.append(i[3])
x = np.array(x)
y = np.array(y)
print("Date : ", x[:10])
print("Response: ", y[:10])

Date :  [[  1 114 105]
 [114 105 573]
 [105 573   4]
 [573   4   5]
 [  4   5 115]
 [  5 115  19]
 [115  19   5]
 [ 19   5 195]
 [  5 195  46]
 [195  46 310]]
Response:  [ 573    4    5  115   19    5  195   46  310 1223]


In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000,activation="relu"))
model.add(Dense(vocab_size,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             78660     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7866)              7873866   
                                                                 
Total params: 21,001,526
Trainable params: 21,001,526
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("next_words.h5", monitor='loss',verbose=1,save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(x,y,epochs=10,batch_size=64,callbacks=[checkpoint])


Epoch 1/10
1097/1097 [==============================] - ETA: 0s - loss: 6.7028
Epoch 1: loss improved from inf to 6.70281, saving model to next_words.h5
1097/1097 [==============================] - 665s 603ms/step - loss: 6.7028
Epoch 2/10
1097/1097 [==============================] - ETA: 0s - loss: 6.2276
Epoch 2: loss improved from 6.70281 to 6.22762, saving model to next_words.h5
1097/1097 [==============================] - 636s 580ms/step - loss: 6.2276
Epoch 3/10
1097/1097 [==============================] - ETA: 0s - loss: 5.8537
Epoch 3: loss improved from 6.22762 to 5.85369, saving model to next_words.h5
1097/1097 [==============================] - 635s 579ms/step - loss: 5.8537
Epoch 4/10
1097/1097 [==============================] - ETA: 0s - loss: 5.5325
Epoch 4: loss improved from 5.85369 to 5.53247, saving model to next_words.h5
1097/1097 [==============================] - 639s 583ms/step - loss: 5.5325
Epoch 5/10
1097/1097 [==============================] - ETA: 0s - loss: 

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl','rb')) 
def Predict(model, tokenizer, text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  pred = np.argmax(model.predict(sequence))
  predicted = ""

  for key, value in tokenizer.word_index.items():
    if value == pred:
      predicted = key
      break
  print(predicted)
  return predicted

In [ ]:
while(True):
  text = input("Enter your line : ")
  if text == "0":
    print("Exceution Completed!!")
  else:
    try:
      text = text.split()
      text = text[-3:]
      print(text)
      Predict(model,tokenizer, text)

    except Exception as e:
      print("Error Occured !! : " , e)
      continue

['The', 'Signora', 'had']
1/1 [==============================] - 1s 1s/step
been
['You', 'may', 'copy']
1/1 [==============================] - 0s 48ms/step
it
